## Imports

In [69]:
# utilities
import pandas as pd
import numpy as np
import os
from IPython.display import Markdown, display
import matplotlib.pyplot as plt
import pandas as pd
import random



# sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix, recall_score, precision_score, accuracy_score
from sklearn.metrics import classification_report
from aif360.metrics import BinaryLabelDatasetMetric

import aif360.sklearn as skm


# aif360
from aif360.sklearn.detectors import bias_scan
# Import necessary modules from aif360
from aif360.metrics import ClassificationMetric
from aif360.datasets import StandardDataset, BinaryLabelDataset


# onnx imports
import onnxruntime as rt
import onnx
from skl2onnx.common.data_types import FloatTensorType
from skl2onnx import to_onnx
from skl2onnx import convert_sklearn

In [146]:
def change_data(train,feature_name : str, is_fraud : bool, value_from : int, value_to : int, num_data_points = 1000):
    is_fraud = 1 if is_fraud else 0
    data = train.copy(deep = True)
    data = data.loc[data['checked'] == is_fraud]
    data = data.loc[data[feature_name] == value_from]

    indexes = data.index.to_numpy()[:num_data_points]
    random.shuffle(indexes)
    
    for i in indexes:
        train.loc[train.index == i, feature_name] = value_to

In [147]:
unprivileged_groups = {'persoon_geslacht_vrouw': 1, 
                        'persoon_leeftijd_bij_onderzoek': 1,
                        'belemmering_ind': 1,
                        'belemmering_ind_hist': 1,
                        'typering_ind': 1,
                        'typering_hist_inburgeringsbehoeftig': 1.0,
                        'persoonlijke_eigenschappen_ind_activering_traject': 1.0,
                        'persoonlijke_eigenschappen_ind_buiten_kantoortijden': 1.0,
                        'persoonlijke_eigenschappen_ind_regulier_arbeidsritme': 1.0}

In [148]:
from scripts import preprocessing


model = GradientBoostingClassifier()

ds_train = pd.read_csv('./../data/train_badly.csv')
ds_test = pd.read_csv('./../data/test.csv')
instance_weights = pd.read_csv('./../data/instance_weights_bad_model.csv')

'''
ds_train['persoon_leeftijd_bij_onderzoek'] = (ds_train['persoon_leeftijd_bij_onderzoek'] <= 27).astype(float)
for key in unprivileged_groups.keys():
    change_data(ds_train, key, False ,unprivileged_groups[key],unprivileged_groups[key] -1 )
    
ds_train.to_csv('train_badly.csv', index=False)
'''
X_train, y_train = preprocessing.preprocess(ds_train)
X_test, y_test = preprocessing.preprocess(ds_test)

model.fit(X_train, y_train, sample_weight=instance_weights.to_numpy().ravel())

y_pred_rew = model.predict(X_test)

results = classification_report(y_test, y_pred_rew)
(tn, fp, fn, tp)  = confusion_matrix(y_test, y_pred_rew).ravel()
print(f"tn: {tn} fp: {fp} fn: {fn} tp: {tp} ")
print(results)


tn: 744 fp: 1516 fn: 7 tp: 262 
              precision    recall  f1-score   support

           0       0.99      0.33      0.49      2260
           1       0.15      0.97      0.26       269

    accuracy                           0.40      2529
   macro avg       0.57      0.65      0.38      2529
weighted avg       0.90      0.40      0.47      2529



In [136]:
def evaluate_model(model, X_test, y_test):
    """Evaluates the model and returns performance metrics

    Args:
        modelTrained model
        X_testTest features
        y_test: Test labels

    Returns:
        Dictionary containing fpr, tnr, npr, fnr, precision, recall, f1
    """
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    tn, fp, fn, tp = cm.ravel()
    print(f'TN {tn}, FP {fp}, FN {fn}, TP {tp}')
    fpr = fp / (fp + tp)  # False Positive Rate
    tnr = tn / (tn + fp)  # True Negative Rate
    tpr = tp / (tp + fn)  # True Positive Rate
    fnr = fn / (fn + tn)  # False Negative Rate
    precision = tp / (tp + fp)  # Precision
    recall = tp / (tp + fn)  # Recall
    f1 = 2 * (precision * recall) / (precision + recall)  # F1 Score

    return {
        "fpr": fpr,
        "tnr": tnr,
        "tpr": tpr,
        "fnr": fnr,
        "precision": precision,
        "recall": recall,
        "f1": f1
    }


In [145]:
evaluate_model(model, X_test,y_test)

TN 744, FP 1516, FN 7, TP 262


{'fpr': 0.8526434195725534,
 'tnr': 0.3292035398230089,
 'tpr': 0.9739776951672863,
 'fnr': 0.009320905459387484,
 'precision': 0.14735658042744657,
 'recall': 0.9739776951672863,
 'f1': 0.25598436736687835}

In [138]:
#code for pipeline
pipeline = Pipeline(steps=[('classification', model)])

In [ ]:
#code for savng training data
ds_train.to_csv('train_badly.csv', index=False)


In [141]:
# Let's convert the model to ONNX
onnx_model = convert_sklearn(
    pipeline, initial_types=[('X', FloatTensorType((None, X_train.shape[1])))],
    target_opset=12)

# Let's check the accuracy of the converted model
sess = rt.InferenceSession(onnx_model.SerializeToString())
y_pred_onnx =  sess.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)

Accuracy of the ONNX model:  0.3977856860419138


In [144]:
# Let's save the model
onnx.save(onnx_model, "./../model/bad_model.onnx")

# Let's load the model
new_session = rt.InferenceSession("./../model/bad_model.onnx")

# Let's predict the target
y_pred_onnx2 =  new_session.run(None, {'X': X_test.values.astype(np.float32)})

accuracy_onnx_model = accuracy_score(y_test, y_pred_onnx2[0])
print('Accuracy of the ONNX model: ', accuracy_onnx_model)


Accuracy of the ONNX model:  0.3977856860419138
